# Wildfire Baseline Modeling — Feature Scaling, Training & Evaluation

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt


1. LOAD/inspect  DATA

In [4]:
data_path = '../data/outputs/combined_dataset.parquet'
df = pd.read_parquet(data_path)

print('Data shape:', df.shape)
print('Columns:', df.columns.tolist())

Data shape: (21132722, 22)
Columns: ['y', 'x', 'band', 'Caldor_dNBR', 'Caldor_SPI', 'Caldor_VCI', 'Caldor_veg_indices', 'spatial_ref', 'fire_name', 'severity', 'Camp_dNBR', 'Camp_SPI', 'Camp_VCI', 'Camp_veg_indices', 'Dixie_dNBR', 'Dixie_SPI', 'Dixie_VCI', 'Dixie_veg_indices', 'Troublesome_dNBR', 'Troublesome_SPI', 'Troublesome_VCI', 'Troublesome_veg_indices']


2. FEATURE / LABEL SPLIT


In [6]:
target_col = 'severity' # Example: 'severity' or 'burned'
feature_cols = [col for col in df.columns if col not in [target_col, 'geometry', 'x', 'y']]


X = df[feature_cols].values
y = df[target_col].values


X = np.nan_to_num(X, nan=0.0)


3. TRAIN / TEST SPLIT

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# --- Feature Normalization ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ValueError: could not convert string to float: 'Troublesome'

# 4. FEATURE NORMALIZATION

In [ ]:
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg.fit(X_train_scaled, y_train)
y_pred_lr = log_reg.predict(X_test_scaled)
y_prob_lr = log_reg.predict_proba(X_test_scaled)[:, 1] if len(np.unique(y)) == 2 else None


# Random Forest
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1] if len(np.unique(y)) == 2 else None


# --- Evaluation ---
print('\n=== Logistic Regression Report ===')
print(classification_report(y_test, y_pred_lr))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_lr))


print('\n=== Random Forest Report ===')
print(classification_report(y_test, y_pred_rf))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_rf))


# --- ROC Curves (Binary Classification Only) ---
if y_prob_lr is not None:
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_prob_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)


plt.figure(figsize=(8,6))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {roc_auc_lr:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.show()
else:
print('\nROC curves not generated (multi-class target detected).')

# 5. BASELINE MODEL — Logistic Regression

In [ ]:
print("\n=== Logistic Regression ===")
log_reg = LogisticRegression(max_iter=2000, class_weight='balanced', multi_class='multinomial', solver='saga')
log_reg.fit(X_train_scaled, y_train)
y_pred_lr = log_reg.predict(X_test_scaled)
print(classification_report(y_test, y_pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))


# 6. BASELINE MODEL — Random Forest

In [ ]:
print("\n=== Random Forest ===")
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))


# 7. FEATURE IMPORTANCE (RF ONLY)

In [ ]:
importances = rf.feature_importances_
idx = np.argsort(importances)[::-1]
top_n = min(10, len(numeric_cols))
plt.figure(figsize=(10, 4))
plt.bar([numeric_cols[i] for i in idx[:top_n]], importances[idx[:top_n]])
plt.xticks(rotation=45, ha='right')
plt.title("Top {} Feature Importances (Random Forest)".format(top_n))
plt.tight_layout()
plt.show()